# Power electricity power consumption prediction model

Let's first start by importing the needed libraries

In [1]:
from pyspark.sql import SparkSession

And create a local parallel spark session

In [2]:
spark = SparkSession.builder \
        .master("local[*]") \
        .appName("Power electricity prediction") \
        .getOrCreate()

The we read the dataframe we need to apply the prediction on

In [3]:
train = spark.read.csv('../data/engineered/powerelectricity_train.csv', header=True, inferSchema=True)
test = spark.read.csv('../data/engineered/powerelectricity_test.csv', header=True, inferSchema=True)

Now we will test some regression models and evaluate them using rmse

In [4]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor, GBTRegressor, LinearRegression
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler

In [5]:
train_ep = train.drop('rms_current')
train_ep = train.drop('electric_power')

#train_rms = train.drop('electric_power')

test_ep = test.drop('rms_current')
test_ep = test.drop('electric_power')

#test_rms = test.drop('electric_power')

features_ep = train_ep.columns.copy()
#features_ep.remove('electric_power')
features_ep.remove('delta_y')

#features_rms = train_rms.columns.copy()
#features_rms.remove('rms_current')

In [6]:
assembler_pe_train = VectorAssembler(inputCols=features_ep,outputCol="features")
train_ep = assembler_pe_train.transform(train_ep)

Let's start by predicting electric_power

In [7]:
rf_pe = RandomForestRegressor(featuresCol="features",labelCol='delta_y',numTrees=20, maxDepth=11,seed=42)

In [8]:
%%time

model_pe = rf_pe.fit(train_ep)

CPU times: user 12.7 ms, sys: 10.8 ms, total: 23.5 ms
Wall time: 20.9 s


In [9]:
assembler_pe_test = VectorAssembler(inputCols=features_ep,outputCol="features")
test_ep = assembler_pe_test.transform(test_ep)

In [10]:
predictions = model_pe.transform(test_ep)

In [11]:
evaluator_rmse = RegressionEvaluator(
    labelCol="delta_y", predictionCol="prediction", metricName="rmse")
rmse = evaluator_rmse.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

evaluator_r2 = RegressionEvaluator(
    labelCol="delta_y", predictionCol="prediction", metricName="r2")
r2 = evaluator_r2.evaluate(predictions)
print("R squared (r2) on test data = %g" % r2)

evaluator_mae = RegressionEvaluator(
    labelCol="delta_y", predictionCol="prediction", metricName="mae")
mae = evaluator_mae.evaluate(predictions)
print("Mean Average Error (RMSE) on test data = %g" % mae)

Root Mean Squared Error (RMSE) on test data = 0.399019
R squared (r2) on test data = 0.932685
Mean Average Error (RMSE) on test data = 0.295191


In [12]:
predictions.select(["delta_y", "prediction"]).show(100000)

+--------------------+--------------------+
|             delta_y|          prediction|
+--------------------+--------------------+
|  0.3831713896574964|  0.3444233417185534|
|  0.5582151680453507|  0.7424847535240586|
|  0.5582151680453507|   0.721671498102382|
|  0.5582151680453507|   0.721671498102382|
|  0.3476407540450346| 0.33417431271279263|
| 0.06735910139058454| 0.17629646750983452|
|  0.1766774216981588|  0.2667583413144044|
| 0.35730156822812376|  0.3014346296593685|
|  0.5643760041147257|  0.5470825858404176|
|  0.8536479705886499|  1.2334258132650477|
|  0.4529802355210659|  0.8258113321759977|
|  0.9974610877921606|  1.0132150624181384|
|  0.5335925996353765|  0.6876074119650409|
|  0.6322382652544452|  0.7891084011567088|
|   1.126864443971563|  1.2357533770037994|
| 0.11070358722432516| 0.28558063340521217|
| 0.46950997077134105| 0.34728152215635644|
| 0.44433379024100006|  0.3607216165439049|
|  0.8483083163917122|  0.7987664276070265|
|  0.7641504408145607|   0.78209